## Step by step yield calculation from WAPOR data

#### Introduction

The waporact package includes a set of statistical functions and visualisation (tools) that can be used to carry out the analysis of any raster or rasters that the user provides. 

In this example we will build on the tutorial:

*waporact\tutorials\01_waporact_basics\01B_basic_statistical_analysis.ipynb*

In this tutorial we will show you how you can take the basic/generic functions
available in: 

*waporact\scripts\tools\statistics.py* <br>
*waporact\scripts\tools\raster.py* <br>
*waporact\scripts\tools\vector.py* <br>

and use them to carry out a yield calculation and finally how to combine the steps taken into a single function for production use.

For details on the theory behind the yield and water productivity calculations being carried out see the wiki:

*https://github.com/eLEAF-Github/WAPORACT/wiki/3.-Theory-2.-Performance-Assessment-Indicators*

specifically sections:

5. Land Productivity (Yield)
6. Crop Water Productivity<br><br>

NOTE: The retrieval steps carried out in this notebook are a copy of those carried out in the notebook: *waporact\tutorials\01_Basics\01A_downloads\01A_downloading_from_wapor.ipynb*<br>

### **Steps**:<br>

1. Importing of the modules and functions needed<br><br> 

2. Retrieve NPP and AETI rasters for a given period

    2.1) retrieve NPP and AETI rasters. <br>

    2.2) check out the file paths in the retrieval dictionary. <br><br> 
    
3. sum the NPP and AETI rasters each across time

    3.1) extract an array from one of the rasters as an example with the function *raster_to_array*. <br>
    
    3.2) sum the AETI and NPP rasters. <br><br> 

4. Calculate Yield and Water Productivity from the summed rasters.  <br><br> 

    4.1) set required user parameters. <br>
    
    4.2) calculate yield. <br>

    4.3) calculate Water Productivity. <br><br>

5. Turn the process into a single function  <br><br>

    5.1) construct the water productivity function. <br>
    
    5.2) run the water productivity function. <br><br> 

NOTE: If this is your first time running this please read the instructions below and follow the steps, otherwise feel free to use the notebook as you wish.
***

In [ ]:
# import the required functions/classes
import os
from datetime import datetime

# import retrieval class
from waporact.scripts.retrieval.wapor_retrieval import WaporRetrieval
print('retrieval class succesfully imported')

# import statistics functions
from waporact.scripts.tools import raster
print('raster functions succesfully imported')
# import statistics functions
from waporact.scripts.tools import statistics
print('statistics functions succesfully imported')

from waporact.scripts.tools import plots
print('plotting functions succesfully imported')

print('all scripts imported successfully, you are at the starting line')

***
## 2. Retrieve NPP and AETI rasters for a given period

***
### 2.1 Retrieve WAPOR NPP and AETI rasters

Retrieve the NPP (Net Primary Porductivity) and AETI (Evapotranspiration) rasters from wapor for your given area. The steps taken below are the same as those described in the tutorial notebook:

 *waporact\tutorials\01_Basics\01A_downloads\01A_downloading_from_wapor.ipynb* <br>

 the only difference is that the datacomponents argument has changed to: **NPP,AETI**

 we also set the period_start and period_end to **2020/1/1 -> 2020/2/1** to make sure of data availability


 The first step of this is initiating the class **WaporRetrieval** and assigning it to an object 

In [ ]:
# activation of the wapor retrieval class 
retrieval = WaporRetrieval(            
    waporact_directory=r'<insert_directory_path_here>',
    shapefile_path=r"<insert_git_directory_path_here>\waporact\samples\shapefile\gezira_test_set.shp",
    wapor_level=3,
    period_start=datetime(2022,1,1),
    period_end=datetime(2022,2,1),
    project_name='calc_yield',
    api_token='<insert_api_toke_here>')

After this you can retrieve the rasters themselves by providing the correct arguments:

In [ ]:
# retrieve the actual rasters
retrieved_rasters = retrieval.download_wapor_rasters(                       
    datacomponents=['NPP','AETI'],                                          
    period_start=datetime(2022,1,1),                                         
    period_end=datetime(2022,2,1),                                          
)

***
### 2.2 check out the file paths in the retrieval dictionary

It is important to note that when rasters are retrieved using **WaporRetrieval** (assigned to the object *retrieval* above) they are automatically assigned to a dictionary structure as the function used *download_wapor_rasters* can be used to download multiple rasters and build vrts so to store them a system is needed this is done by returning a dictionary:


for detials on the structure see the function info for *download_wapor_rasters* in either:

waporact\scripts\retrieval\wapor_retrieval.py

or online at:

https://github.com/eLEAF-Github/WAPORACT/wiki/2.-The-WaPORAct-Package-3.-Function-and-Class-Descriptions-1.-retrieval

below follows an example on how to access the dictionary produced that contains the file paths:

In [ ]:
# print the retrieved_rasters object
print('retrieved_rasters dict::\n {} \n'.format(retrieved_rasters))

# print the list of retrieved rasters for NPP
print('retrieved NPP rasters:\n {} \n'.format(retrieved_rasters['NPP']['raster_list']))

# print the path to the NPP vrt
print('path to the retrieved NPP vrt:\n {}'.format(retrieved_rasters['NPP']['vrt_path']))

# print the list of retrieved rasters for AETI
print('retrieved AETI rasters:\n {} \n'.format(retrieved_rasters['AETI']['raster_list']))

# print the path to the AETI vrt
print('path to the retrieved AETI vrt:\n {}'.format(retrieved_rasters['AETI']['vrt_path']))

***
## 3. Sum the NPP and AETI rasters each across time 

This step is being carried out as the seasonal sum of NPP and AETI is required to calculate Yield and Water Productivity

***
### 3.1 extract an array from one of the rasters as an example with the function *raster_to_array*.

As an example before calculating the sum of NPP and AETI across time we wnat to show you how to extract an array from a raster for calculations using the function *raster_to_array*. 

This is to show you <br><br>

A) one of the basic functions on which this whole package is built <br><br>

B) what is available in this package if you know where to look/ do a little searching.

We call the function *raster_to_array* from the previously imported raster script.

earlier cell:

*from waporact.scripts.tools import raster*

So all raster functions are now available for use after this import simply by adding '.' and then the name of the function you want to use.

Below we extract the array of one of the retrieved rasters and print it as an example. For most raster calculations the first ste is getting access to the array inside

In [ ]:
# extract the array
npp_array= raster.raster_to_array(input_raster_path=retrieved_rasters['NPP']['raster_list'][0])

#print the array
print(npp_array)

# do not be alarmed if you see alot of nan's it is likely that this is simply due to the raster being masked

The other half of this process is writing an array to raster and this is achieved in WaPORAct usign the function: *array_to_raster*

***
### 3.2 sum the AETI and NPP rasters

Now that we have seen how to extract a raster we are going to use a more complex function that is built around *raster_to_array* and *array_to_raster* and other subfunctions to calculate the sum of AETI and NPP. This is done using the function

*calc_multiple_array_numpy_statistic*

from the statistics script

for details on this function please see the script:*waporact\scripts\tools\statistics.py* or the wiki page: *https://github.com/eLEAF-Github/WAPORACT/wiki/2.-The-WaPORAct-Package-3.-Function-and-Class-Descriptions-6.-statistics*

this function is used to calculate/apply a numpy statistical function across multiple input rasters given as either a list or a vrt.

In [ ]:
# import the numpy package so that the numpy function can be applied
import numpy as np

print('numpy imported')

In [ ]:
# set the output path for the AETI sum raster
aeti_sum_raster = r'<insert_chosen_directory_path_here>\\L3_AETI_sum.tif'

# calculate the sum of seasonal AETI raster
aeti_sum_raster = statistics.calc_multiple_array_numpy_statistic(
    input=retrieved_rasters['AETI']['raster_list'], #provide the list of input rasters
    numpy_function=np.nansum, # apply the numpy function
    axis=0, # set the axis on which to apply the function 0=z (see script function for details)
    output_raster_path=aeti_sum_raster) # set an output path for the created sum raster

print('aeti sum raster made: {}'.format(aeti_sum_raster))

In [ ]:

# set the output name and path of the output NPP sum raster
npp_sum_raster = r'<insert_chosen_directory_path_here>\\L3_NPP_sum.tif'

# calculate the sum of seasonal NPP raster
npp_sum_raster = statistics.calc_multiple_array_numpy_statistic(
    input=retrieved_rasters['NPP']['raster_list'], #provide the list of input rasters
    numpy_function=np.nansum, # apply the numpy function
    axis=0, # set the axis on which to apply the function 0=z (see script function for details)
    output_raster_path=npp_sum_raster) # set an output path for the created sum raster


print('npp sum raster made: {}'.format(npp_sum_raster))

***
## 4. Calculate Yield and Water Productivity from the summed rasters

with access to the sum of AETI and NPP rasters yield and WP (Water Productivity) can be calculated

***
### 4.1 set required user parameters

to calculate yield and WP certain user defined parameters need to be set. 

For details on what the below parameters mean please again see:

*https://github.com/eLEAF-Github/WAPORACT/wiki/3.-Theory-2.-Performance-Assessment-Indicators*

specifically sections:

5. Land Productivity (Yeild)
6. Crop Water Productivity<br><br>

In [ ]:
# set the crop parameters 
AOT = 0.8
HI = 0.20
mc = 0.15
C4 = 1

print('parameters set')

***
### 4.2 calculate yield

to calculate yield we extract the array from the sum_npp raster calculated above
apply the required formula and write the resulting array to raster again.

In [ ]:
# yield calculation 

# extract the npp array
npp_sum_array = raster.raster_to_array(input_raster_path=npp_sum_raster)

# calculate the yield array
yield_array = ((npp_sum_array*22.222*10*HI*C4)/(1-mc))*0.1

# set the output path for the yield raster
yield_raster = r'<insert_chosen_directory_path_here>\\L3_Yield.tif'

# output the yield array to raster
yield_raster = raster.array_to_raster(
    metadata=npp_sum_raster, # the metadata argument allows you to use the metadata of an existing raster as metadata for the new one
    output_raster_path=yield_raster, 
    input_array=yield_array)


print('yield raster made: {}'.format(yield_raster))

***
### 4.3 calculate Water Productivity

In [ ]:
# wp calculation 

# extract the aeti array
aeti_sum_array = raster.raster_to_array(input_raster_path=aeti_sum_raster)

# extract the yield array
yield_array = raster.raster_to_array(input_raster_path=yield_raster)

# calculate the wp array
wp_array = yield_array / aeti_sum_array

# set the output path for the wp raster
wp_raster = r'<insert_chosen_directory_path_here>\\L3_wp.tif'

# output the yield array to raster
wp_raster = raster.array_to_raster(
    metadata=aeti_sum_raster, # the metadata argument allows you to use the metadata of an existing raster as metadata for the new one
    output_raster_path=wp_raster, 
    input_array=wp_array)

print('water productivity raster made: {}'.format(wp_raster))


### 4.4 output water productivity map

as a last step we produce a water productivity map of the result produced above. This is done using the functions available 
via *waporact.scripts.tools.plots* , imported at the beginning of the notebook as plots. We will beusing the functions 

*shapeplot* and *interactive_choropleth_map*

For more details on how to use these see the previous notebook: **01B_basic_statistical_analysis**

***
## 5. Turn the process into a single function 

as a last step we provide an example of how to turn all the above steps into a single function. In this way terh calculation becomes more easily reproduced and stable. Please take a look below if you wish

***
### 5.1 construct the water productivity function

In [ ]:
# water productivity function

# import the required functions/classes
import os
from datetime import datetime
import numpy as np
from waporact.scripts.retrieval.wapor_retrieval import WaporRetrieval
from waporact.scripts.tools import raster
from waporact.scripts.tools import statistics

# define the function
def calc_water_productivity(
    waporact_directory: str,
    shapefile_path: str,
    wapor_level: int,
    project_name: str,
    api_token: str,
    start_of_season: datetime,
    end_of_season: datetime,
    return_period: str,
    HI: float,
    mc: float,
    C4: float,
    output_map: bool= True,
    ): # all arguments used in the process above that cannot be automated become arguments in the function
    """
    Description:
        function to calculate yield and water productivity for a given area 
        and season using wapor data

        NOTE:  to keep the function simple we limited the 
        automation of the filenames if this was part of a WaPORAct pipeline 
        we would set the filenames using  the functions in 
        waporact\scripts\structure\wapor_structure.py 
        specifically the function: generate_output_file_path.

    Args:
        waporact_directory: base directory for the project
        shapefile_path: path to the shapefile to use
        wapor_level: wapor data level to analyse
        project_name: name of the project
        return_period: interval to retrieve wapor data for
        api_token: wapor api token used to retrieve data
        start_of_season: date time boject definining the start of season (growth start)
        end_of_season: datetime object defining the end of season (harvest date)
        HI: dry basis harvest index (fraction)
        mc: wet basis grain moisture content (fraction)
        C4: a factor for conversion from C3 to C4 plant (photosynthesis type)
        output_map: if true outputs a html map to folder

    Return:
        list: list holding the paths to the two rasters yield and water productivity
    """
    # set the base file path using waporact_directory

    output_dir = os.path.join(waporact_directory,project_name,'wp_calc')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # set output file paths:
    aeti_sum_raster = os.path.join(output_dir,'AETI_sum.tif')
    npp_sum_raster = os.path.join(output_dir,'NPP_sum.tif')
    yield_raster = os.path.join(output_dir,'Yield.tif.tif')
    water_productivity_raster = os.path.join(output_dir,'water_productivity.tif')

    # activation of the wapor retrieval class 
    retrieval = WaporRetrieval(            
        waporact_directory=waporact_directory,
        shapefile_path=shapefile_path,
        wapor_level=wapor_level,
        period_start =start_of_season,
        period_end =end_of_season,
        return_period=return_period,
        project_name=project_name,
        api_token=api_token)

    # retrieve the npp and aeti rasters
    retrieved_rasters = retrieval.download_wapor_rasters(                       
        datacomponents=['NPP','AETI'],                                          
        period_start=start_of_season,                                         
        period_end=end_of_season,                                          
        )

    # calculate the sum of seasonal AETI raster
    aeti_sum_raster = statistics.calc_multiple_array_numpy_statistic(
        input=retrieved_rasters['AETI']['raster_list'],
        numpy_function=np.nansum,
        axis=0, 
        output_raster_path=aeti_sum_raster)

    # calculate the sum of seasonal NPP raster
    npp_sum_raster = statistics.calc_multiple_array_numpy_statistic(
        input=retrieved_rasters['NPP']['raster_list'], 
        numpy_function=np.nansum,
        axis=0, 
        output_raster_path=npp_sum_raster) 

    # extract the npp array
    npp_sum_array = raster.raster_to_array(input_raster_path=npp_sum_raster)

    # calculate the yield array
    yield_array = ((npp_sum_array*22.222*10*HI*C4)/(1-mc))*0.1 

    # calculate the wp array
    wp_array = yield_array / aeti_sum_array

    # NOTE: Since we already had the yield array we do not need to extract as we did above,
    # this whole function could also be written without writing any intermediates to file

    # output the yield array to raster
    yield_raster = raster.array_to_raster(
        metadata=npp_sum_raster, 
        output_raster_path=yield_raster, 
        input_array=yield_array)
    
    print('yield raster produced: {}'.format(yield_raster))

    # output the yield array to raster
    water_productivity_raster = raster.array_to_raster(
        metadata=aeti_sum_raster, # the metadata argument allows you to use the metadata of an existing raster as metadata for the new one
        output_raster_path=water_productivity_raster, 
        input_array=wp_array)
    
    print('water productivity raster produced: {}'.format(yield_raster))    

    return [yield_raster, water_productivity_raster]

print('we built a function :)')


***
### 5.2 run the water productivity function

In [ ]:
# run the water productivity function made
wp_rasters = calc_water_productivity(
    waporact_directory=r'<insert_directory_path_here>',
    shapefile_path=r"<insert_git_directory_path_here>\waporact\samples\shapefile\gezira_test_set.shp",
    wapor_level=3,
    project_name='calc_yield',
    api_token='<insert_api_toke_here>',
    start_of_season=datetime(2020,1,1),
    end_of_season=datetime(2020,2,1),
    return_period='D',
    HI=0.20,
    mc=0.15,
    C4=1,
    output_map= True,
    )

# print outputted rasters
print('yield and water productivity rasters produced:')
print(wp_rasters)

***
## The next step: Performance Assessment Indicators (PAIs)

f you feel like it you can also take a look at notebook *02A_waporact_calculating_PAIs.ipynb* where we walk you through the process of producing more diffcult statistics: *Performance Assessment Indicators (PAIs)* for an area from download to analysis.